In [16]:
from data_load import load_games_with_tasks, load_games_with_tasks, load_system_prompt, load_games
from utils import get_model_response, GPT_model, get_message_with_example, get_desc_response
import numpy as np
import pandas as pd
import os
import random

# seed random
random.seed(42)

# create directory for saving results
result_dir = f"{GPT_model}_selffed_results"
description_dir = f"{result_dir}/descriptions"
move_dir = f"{result_dir}/moves"
query_dir = f"{result_dir}/query"
for directory in [result_dir, description_dir, move_dir, query_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)


In [2]:
games_with_task = {game.tag: game for game in load_games_with_tasks()}

# Getting the description

In [3]:
desc_training_games = list(filter(lambda x: x.tag not in games_with_task, load_games()))
# take random 10 games for description training
desc_examples = random.sample(desc_training_games, 10)

In [4]:
for game_tag, game in games_with_task.items():
    predicted_code = get_desc_response(get_message_with_example, game, desc_examples)
    game.self_fed_code = predicted_code # we save the predicted code in the game object
    # save generated description in a file
    with open(f"{description_dir}/{game_tag}.txt", "w") as f:
        f.write(predicted_code)

# Getting the code of the moves

In [5]:
def get_self_fed_move_message(game, game_move_list, example_game, example_task_id):
    messages = []

    # put example desc, code, moves to train the model
    messages.append({"role": "system", "content": load_system_prompt('desc_prompt.txt')})
    messages.append({"role": "user", "content": f"Name: {example_game.name}\nDescription: {example_game.description}"})
    messages.append({"role": "assistant", "content": example_game.code})

    # put example moves
    example_move_list = example_game.tasks[example_task_id]['query_list']
    messages.append({"role": "system", "content": load_system_prompt('config_prompt.txt')})
    for move in example_move_list:
        messages.append({"role": "user", "content": move.text})
        messages.append({"role": "assistant", "content": move.code})

    
    # now put the actual desc, code, moves to get the response
    messages.append({"role": "system", "content": load_system_prompt('desc_prompt.txt')})
    messages.append({"role": "user", "content": f"Name: {game.name}\nDescription: {game.description}"})
    messages.append({"role": "assistant", "content": game.self_fed_code})
    messages.append({"role": "system", "content": load_system_prompt('config_prompt.txt')})
    for move in game_move_list[:-1]: # this will be updated as we get responses
        messages.append({"role": "user", "content": move.text}) 
        messages.append({"role": "assistant", "content": move.code}) 

    # final message to get the response, last move is the one we want to predict
    messages.append({"role": "user", "content": game_move_list[-1].text})

    return messages



In [6]:
def get_self_fed_move_responses_task(game, task_id, message_fn):
    task = game.tasks[task_id]
    actual_move_list = task['query_list']
    TOTAL_MOVES = len(actual_move_list)

    self_fed_move_list = []

    responses = []
    answers = []
    query = []
    for move_num in range(TOTAL_MOVES):
        expected_move = actual_move_list[move_num]
        answers.append(expected_move.code)
        expected_move.code = ""
        self_fed_move_list.append(expected_move)
        model_response = get_model_response(message_fn(game, self_fed_move_list))
        responses.append(model_response)
        query.append(expected_move.text)

        self_fed_move_list[-1].code = model_response

    # make dataframe
    df = pd.DataFrame({
        'responses': responses,
        'answers': answers,
        'query': query,
        'game_name': game.tag,
        'task_id': task_id
    })

    task['self_fed_list'] = self_fed_move_list

    return df

def get_move_responses_df(game, message_fn):
    TOTAL_TASKS = len(game.tasks)
    dfs = []
    dfs = [get_self_fed_move_responses_task(game, task_id, message_fn) for task_id in range(TOTAL_TASKS)]
    df = pd.concat(dfs)
    return df

In [7]:
# use doguti as training data for moves
doguti = games_with_task.pop('doguti')

move_message_fn = lambda game, move_list: get_self_fed_move_message(game, move_list, doguti, 0)

In [10]:
# get responses for all games
for game_tag, game in games_with_task.items():
    df = get_move_responses_df(game, move_message_fn)
    df.to_csv(f"{move_dir}/{game_tag}.csv", index=False)
    print(f"Saved {game_tag} responses")

Saved asimplegame responses
Saved tictactoe3d responses
Saved achi responses


# Getting the answer to queries

In [13]:
def get_self_fed_query_messages(game, task_id):
    messages = []
    task = game.tasks[task_id]

    messages.append({"role": "system", "content": load_system_prompt('desc_prompt.txt')})
    messages.append({"role": "user", "content": f"Name: {game.name}\nDescription: {game.description}"})
    messages.append({"role": "assistant", "content": game.self_fed_code})

    
    messages.append({"role": "system", "content": load_system_prompt('config_prompt.txt')})

    for move in task['self_fed_list']:
        messages.append({"role": "user", "content": move.text})
        messages.append({"role": "assistant", "content": move.code})

    messages.append({"role": "system", "content": load_system_prompt('query_prompt.txt')})
    messages.append({"role": "user", "content": task['query']})

    return messages

def get_task_response_df(game):
    model_response = []
    answers = []
    queries = []
    task_ids = []
    for task_id in range(len(game.tasks)):
        messages = get_self_fed_query_messages(game, task_id)
        model_response.append(get_model_response(messages))
        answers.append(game.tasks[task_id]['answer'])
        queries.append(game.tasks[task_id]['query'])
        task_ids.append(task_id)

    df = pd.DataFrame({"responses": model_response, "answers": answers,
                          "queries": queries})
                                 
    return df



In [17]:
for game_tag, game in games_with_task.items():
    df = get_task_response_df(game)
    df.to_csv(f"{query_dir}/{game_tag}.csv", index=False)
    print(f"Saved {game_tag} responses")

Saved asimplegame responses
Saved tictactoe3d responses
Saved achi responses


KeyError: 'self_fed_list'